In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [49]:
import transducer.functional
from transducer.eager import transduce
from transducer.functional import compose
from transducer.reducers import appending
from transducer.transducers import mapping, filtering, reducing, scanning

from transducer.infrastructure import Reduced, Transducer

In [44]:
class Partitioning(Transducer):
    def __init__(self, reducer, predicate):
        super().__init__(reducer)
        self._predicate = predicate
        self._stored = []
    def step(self, result, item):
        if self._predicate(item): # Encountered delimiter
            # yield stored sequence
            if self._stored:
                rv = self._reducer(result, self._stored)
            else:
                rv = result
            # start new sequence
            self._stored = [item]
            return rv
        else: # Encountered non-delimiter item
            # append to existing sequence
            self._stored.append(item)
            return result
    def complete(self, result):
        if self._stored:
            result = self._reducer.step(result, self._stored)
        return self._reducer.complete(result)
    
def partitioning(predicate):
    """If predicate returns true, a new sequence (list) will be started with that item"""
    def partitioning_transducer(reducer):
        return Partitioning(reducer, predicate)

    return partitioning_transducer

print(transduce(partitioning(lambda x:x==0), appending(), [1,3,9,0,2,0,0,1,2,3]))
print(transduce(partitioning(lambda x:x==0), appending(), []))

[[1, 3, 9], [0, 2], [0], [0, 1, 2, 3]]
[]


In [45]:
with open('partition_test.pltme') as f:
    s = f.read()
    print(s)

STARTPATH;EOFILL;0;0;0
MOVETO;59.1836;99.1797
LINETO;5.73438;7.13281
LINETO;112.176;6.87109
LINETO;59.1836;99.1797
MOVETO;43.2422;62.5703
LINETO;74.668;62.5703
LINETO;74.668;56.5703
LINETO;43.2422;56.5703
LINETO;59.1836;99.1797
MOVETO;80.8789;50.2539
LINETO;59.2227;14.1797
LINETO;38.8125;49.5352
LINETO;59.1836;99.1797
MOVETO;80.8789;50.2539
STARTPATH;STROKE;1;0;0
MOVETO;59.1836;99.1797
LINETO;5.73438;7.13281
LINETO;112.176;6.87109
LINETO;59.1836;99.1797
MOVETO;43.2422;62.5703
LINETO;74.668;62.5703
LINETO;74.668;56.5703
LINETO;43.2422;56.5703
LINETO;59.1836;99.1797
MOVETO;80.8789;50.2539
LINETO;59.2227;14.1797
LINETO;38.8125;49.5352
LINETO;59.1836;99.1797
MOVETO;80.8789;50.2539



In [76]:
class PltmePath(object):
    __slots__ = ['coordinates']

    def __init__(self, coordinates):
        self.coordinates = coordinates
        
    def __str__(self):
        return 'Path ' + ', '.join(str(c) for c in self.coordinates)


class PltmePathGroup(object):
    __slots__ = ['paths', 'style', 'color']

    def __init__(self, paths, style, color):
        self.paths = paths
        self.style = style
        self.color = color
        
    def __str__(self):
        s = ['Path group: style={}, color={}'.format(self.style, self.color)]
        s+= ['{} paths:'.format(len(self.paths))]
        s+= ['\t'+str(p) for p in self.paths]
        return '\n'.join(s)
        
    __repr__=__str__

In [77]:
def parse_path(lines):
    def coords(s):
        cmd, x, y = s.split(';')
        return float(x), float(y)
    coords = [ coords(l) for l in lines ]
    return coords

def parse_group(lines):
    cmd, path_type, r, g, b = lines[0].split(';')
    color = float(r), float(g), float(b)
    strokes = transduce(
        compose(
            partitioning(lambda s: s.upper().startswith('MOVETO;')),
            filtering(lambda lines: len(lines)>1),
            mapping(parse_path)
        ),      
        appending(), 
        lines[1:])
    path = PltmePathGroup(strokes, path_type, color)
    return path

transduce(
    compose(
        mapping(lambda s: s.strip()),
        partitioning(lambda s: s.upper().startswith('STARTPATH;')),
        mapping(parse_group)
    ), 
    appending(), 
    s.splitlines())

[Path group: style=EOFILL, color=(0.0, 0.0, 0.0)
 3 paths:
 	[(59.1836, 99.1797), (5.73438, 7.13281), (112.176, 6.87109), (59.1836, 99.1797)]
 	[(43.2422, 62.5703), (74.668, 62.5703), (74.668, 56.5703), (43.2422, 56.5703), (59.1836, 99.1797)]
 	[(80.8789, 50.2539), (59.2227, 14.1797), (38.8125, 49.5352), (59.1836, 99.1797)],
 Path group: style=STROKE, color=(1.0, 0.0, 0.0)
 3 paths:
 	[(59.1836, 99.1797), (5.73438, 7.13281), (112.176, 6.87109), (59.1836, 99.1797)]
 	[(43.2422, 62.5703), (74.668, 62.5703), (74.668, 56.5703), (43.2422, 56.5703), (59.1836, 99.1797)]
 	[(80.8789, 50.2539), (59.2227, 14.1797), (38.8125, 49.5352), (59.1836, 99.1797)]]